<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="Logo HEIG-VD" style="width: 80px;" align="right"/>

# Cours APN - Labo 4 : Visualisation de vecteurs de mots

## Résumé
Le but de ce laboratoire est de visualiser des vecteurs de mots grâce à des méthodes de réduction de dimensionnalité.  La visualisation permettra de deviner plus facilement le "mot du jour" dans le jeu en ligne [Cémantix](https://cemantix.certitudes.org/).

## 1. Prise en main de word2vec

Le modèle Word2vec contient des _embeddings_ des mots qui sont appris automatiquement à partir de textes.  Chaque mot est donc représenté par un vecteur dans un espace avec plusieurs centaines de dimensions (p.ex. 300 ou 500).  Dans cet espace, les mots de sens ou d'usage proches ont des vecteurs proches (les _embeddings_ capturent la similarité entre mots).  Mais la dimensionnalité de l'espace fait qu'il est difficile de visualiser les vecteurs de mots.

La librairie [Gensim](https://radimrehurek.com/gensim/index.html) permet de charger un modèle Word2vec existant (avec la fonction `load_word2vec_format(...)`) et de manipuler les vecteurs de mots (instances de la classe `KeyedVectors`).  On peut notamment utiliser la fonction `similarity(w1, w2)` qui retourne le cosinus des vecteurs correspondant aux mots `w1` et `w2`, à condition que ceux-ci soient connus du modèle.  On peut aussi utiliser la fonction `most_similar([w])` qui retourne les mots voisins d'un ou plusieurs mots.  La [documentation de KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) fournit des exemples utiles.

Veuillez télécharger l'_un des deux_ modèles word2vec suivants, déjà entraînés, mis à disposition par [J.-Ph. Fauconnier](https://fauconnier.github.io/#data):
- [frWac_no_postag_no_phrase_500_cbow_cut100.bin](https://embeddings.net/embeddings/frWac_no_postag_no_phrase_500_cbow_cut100.bin) - plus petit (229 Mo), mais pas identique au modèle utilisé pour Cémantix ;
- [frWac_no_postag_phrase_500_cbow_cut10.bin](https://embeddings.net/embeddings/frWac_no_postag_phrase_500_cbow_cut10.bin) - bien plus grand (2 Go) donc assez gourmand en mémoire, mais qui est exactement celui utilisé pour Cémantix.
Veuillez placer le modèle choisi dans le dossier `gensim-data` de votre dossier utilisateur.

In [2]:
# Téléchargement, si nécessaire.
# import wget
# url = 'https://embeddings.net/embeddings/frWac_no_postag_no_phrase_500_cbow_cut100.bin'
# wget.download(url, path_to_model)

In [3]:
from gensim.models import KeyedVectors

In [4]:
path_to_model = "C:\\Users\\Julien\\Documents\\HEIG-VD\\S5_2024\\APN\\APN\\lab4\\frWac_no_postag_phrase_500_cbow_cut10.bin" # à adapter

In [5]:
model = KeyedVectors.load_word2vec_format(path_to_model, binary=True, unicode_errors="ignore")

a. Choisissez deux mots de sens proches `w1` et `w2`, et un autre plus différent noté `w3`. Affichez la similarité selon word2vec (cosinus) entre chaque paire de mots.  Ces valeurs correspondent-elles à vos intuitions ?

In [6]:
w1 = 'chat'
w2 = 'félin'
w3 = 'lapper'

sim_w1_w2 = model.similarity(w1, w2)
sim_w1_w3 = model.similarity(w1, w3)
sim_w2_w3 = model.similarity(w2, w3)

print(f"Similarité entre '{w1}' et '{w2}': {sim_w1_w2:.4f}")
print(f"Similarité entre '{w1}' et '{w3}': {sim_w1_w3:.4f}")
print(f"Similarité entre '{w2}' et '{w3}': {sim_w2_w3:.4f}")


Similarité entre 'chat' et 'félin': 0.5748
Similarité entre 'chat' et 'lapper': 0.1546
Similarité entre 'félin' et 'lapper': 0.1847


b. Affichez les 10 mots les plus proches de `w1` selon word2vec avec pour chacun sa similarité avec `w1`. 

In [7]:
w1 = 'chat'
#mots voisins
neighbour_words = model.most_similar(w1, topn=10)
print(f"Les 10 mots les plus proches de '{w1}")

for word, similarity in neighbour_words:
    print(f"{word}: {similarity:.4f}")

Les 10 mots les plus proches de 'chat
minou: 0.6335
matou: 0.6266
petit_chatte: 0.6141
chatte: 0.5922
miauler: 0.5909
chaton: 0.5889
quoi_fouetter: 0.5832
félin: 0.5748
chien: 0.5650
chats: 0.5555


c. Quels sont les 15 premiers coefficients du vecteur (_embedding_) du mot `w1` ? Quelle est la dimension de ce vecteur ?  Quelle est la taille du vocabulaire connu du modèle ?  Vous pouvez simplement écrire les commandes répondant à ces questions.

In [8]:
w1 = 'chat'
print(f"Les 15 premiers coefficients pour '{w1}':")
print(model[w1][:15])

print(f"La dimension du vecteur'{w1}': {len(model[w1])}")

print(f"Taille du vocabulaire du modèle : {len(model.index_to_key)}")


Les 15 premiers coefficients pour 'chat':
[-0.5510563  -0.22379288  0.72409904 -0.82713395 -0.68234926  2.0828469
 -0.30507645  0.400353    0.9901076  -1.2962811   1.2319442   1.5554144
  0.3478864  -1.6153904  -0.585831  ]
La dimension du vecteur'chat': 500
Taille du vocabulaire du modèle : 1081995


d. Veuillez écrire une fonction appelée `neighbors` selon les spécifications suivantes.  Cette fonction servira plus loin pour l'affichage.
* input : modèle, liste de mots, nombre de voisins (`topn`) ;
* output : liste de mots voisins de chacun des mots de l'input, représentés par un dictionnaire expliqué ci-après ;
* fonctionnement : pour _chacun_ des mots donnés en input, la fonction teste si le mot est dans le vocabulaire du modèle word2vec (sinon elle ne le considère pas), puis demande au modèle la liste des `topn` mots voisins ; pour chacun de ces mots, la fonction construit un `dict` à 4 champs : 
  - mot voisin
  - similarité
  - mot de départ
  - code de couleur associé au mot de départ : 1, 2, etc.

**Exemple** : si on appelle la fonction avec \['école'\], le début du résultat sera :
```
[{'neighbor': 'scolaire',
  'similarity': 0.7114928364753723,
  'ref_word': 'école',
  'color_code': 1}, ...
```

In [9]:
def neighbors(model, word_list, topn = 5):
    results = []
    for idx, word in enumerate(word_list):
        if word in model:

            similar_words = model.most_similar(word, topn=topn)
            
            for neighbor, similarity in similar_words:
                results.append({
                    'neighbor': neighbor,
                    'similarity': similarity,
                    'ref_word': word,
                    'color_code': idx + 1})
        else:
            print(f"Le mot '{word}' n'est pas dans le vocabulaire du modèle.")
    
    return results

In [10]:
# Tester avec cet appel :
print(neighbors(model, ['chat', 'étudier'], 3))

[{'neighbor': 'minou', 'similarity': 0.6334686875343323, 'ref_word': 'chat', 'color_code': 1}, {'neighbor': 'matou', 'similarity': 0.626587450504303, 'ref_word': 'chat', 'color_code': 1}, {'neighbor': 'petit_chatte', 'similarity': 0.6141418218612671, 'ref_word': 'chat', 'color_code': 1}, {'neighbor': 'étude', 'similarity': 0.6593688726425171, 'ref_word': 'étudier', 'color_code': 2}, {'neighbor': 'analyser', 'similarity': 0.6340749263763428, 'ref_word': 'étudier', 'color_code': 2}, {'neighbor': 'approfondir', 'similarity': 0.5609511733055115, 'ref_word': 'étudier', 'color_code': 2}]


## 2. Affichage simple des voisins d'un mot

Pour commencer, veuillez étudier et exécuter le code fourni en exemple : `Labo4_plotly.ipynb`.  Cela vous montrera comment utiliser l'affichage 2D/3D avec `plotly.js`, et vous permettra de traiter les questions suivantes.

Veuillez écrire et exécuter une fonction qui effectue les opérations suivantes, étant donné un mot en entrée :
* obtenir les vecteurs word2vec du mot et de ses `topn` plus proches voisins ;
* transformer ces vecteurs en 2D par l'ACP ;
* afficher en 2D chaque points correspondant à un mot, avec comme étiquette le mot lui-même.

**Notes:** cette fonction sert de version préliminaire à une fonction plus générique demandée ci-dessous.  Il n'est pas demandé de distinguer le mot entré des mots voisins, dans l'affichage (même type de points).  Il n'est pas nécessaire pour l'instant d'utiliser la fonction `neighbors`.

In [11]:
import plotly
import numpy as np
import plotly.graph_objs as go
from sklearn.decomposition import PCA

In [12]:
def display_PCA_2D_neighbors(model, word, topn = 5):
    if word not in model:
        print(f"Le mot '{word}' n'est pas dans le vocabulaire du modèle.")
        return
    
    similar_words = model.most_similar(word, topn=topn)
    words = [word] + [w[0] for w in similar_words]  # Liste de mots incluant le mot de départ
    vectors = np.array([model[w] for w in words])   # Récupérer les vecteurs correspondants
    
    pca = PCA(n_components=2)
    vectors_2d = pca.fit_transform(vectors)
    
    trace = go.Scatter(
        x=vectors_2d[:, 0],
        y=vectors_2d[:, 1],
        mode='markers+text',
        text=words,
        textposition='top center',
        marker=dict(size=10, color='blue')
    )
    layout = go.Layout(
        title=f"Projection 2D de '{word}' et de ses voisins",
        xaxis=dict(title='CP 1'),
        yaxis=dict(title='CP 2'),
        showlegend=False
    )

    fig = go.Figure(data=[trace], layout=layout)
    fig.show()

In [13]:
display_PCA_2D_neighbors(model, 'chat', 20)

## 3. Affichage configurable des voisins de plusieurs mots

Le but maintenant est d'écrire une fonction `display_dimred_neighbors` qui affiche sur un seul graphique les mots voisins de chaque mot d'une liste de mots donnés.  Vous utiliserez ici la fonction auxiliaire `neighbors` définie plus haut  Les paramètres suivants seront passés à la fonction `display_dimred_neighbors` :

* `model` - le nom du modèle word2vec
* `word_list` - liste de mots dont on veut afficher les voisins (s'ils existent dans `model`)
* `n_components` - dimensionnalité de l'affichage, 2 ou 3
* `topn` - nombre de voisins à afficher pour chaque mot 
* `method` - **méthode de réduction de dimensionnalité : pca, mds, isomap, tsne, ou umap**
* `n_neighbors`- nombre de voisins considérés par la méthode (applicable à isomap, umap, et tsne (appelé alors `perplexity`)).

Trois tests sont demandés à la question 4, mais vous pouvez en exécuter d'autres et les inclure ici.

In [14]:
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.manifold import Isomap
from sklearn.manifold import TSNE
from umap import UMAP # Attention, installer le module nommé 'umap-learn' (avec conda install).

In [62]:
def display_dimred_neighbors(model, word_list, n_components = 3, topn = 5, method = 'pca', n_neighbors = 5):
    # Obtenir les voisins pour chaque mot à partir de la fonction `neighbors`
    all_neighbors = neighbors(model, word_list, topn=topn)
    
    # Vérifier si des voisins ont été trouvés
    if not all_neighbors:
        print("Aucun mot voisin n'a été trouvé.")
        return
    
    # Récupérer les vecteurs et les étiquettes
    words = [entry['neighbor'] for entry in all_neighbors] + [entry['ref_word'] for entry in all_neighbors]
    unique_words = list(set(words))  # Eliminer les doublons
    vectors = np.array([model[word] for word in unique_words])

    
    # Choisir la méthode de réduction de dimensionnalité
    if method == 'pca':
        reducer = PCA(n_components=n_components)
    elif method == 'mds':
        reducer = MDS(n_components=n_components, n_init=4, max_iter=300)
    elif method == 'isomap':
        reducer = Isomap(n_components=n_components, n_neighbors=n_neighbors)
    elif method == 'tsne':
        reducer = TSNE(n_components=n_components, perplexity=n_neighbors, n_iter=500, random_state=42)
    elif method == 'umap':
        reducer = UMAP(n_components=n_components, n_neighbors=n_neighbors, random_state=42)
    else:
        raise ValueError("Méthode de réduction de dimensionnalité non reconnue.")
    
    # Réduction des vecteurs en n dimensions
    vectors_reduced = reducer.fit_transform(vectors)
    
    # Création des traces pour l'affichage
    if n_components == 2:
        trace = go.Scatter(
            x=vectors_reduced[:, 0],
            y=vectors_reduced[:, 1],
            mode='markers+text',
            text=unique_words,
            marker=dict(size=10, color='rgba(93, 164, 214, 0.8)'),
            textposition='top center'
        )
    elif n_components == 3:
        trace = go.Scatter3d(
            x=vectors_reduced[:, 0],
            y=vectors_reduced[:, 1],
            z=vectors_reduced[:, 2],
            mode='markers+text',
            text=unique_words,
            marker=dict(size=5, color='rgba(255, 0, 0, 0.8)'),
            textposition='top center'
        )
    
    # Définir le layout
    layout = go.Layout(
        title=f"Projection {n_components}D des voisins de {', '.join(word_list)} par {method.upper()}",
        width=1200,
        height=1000,
        scene=dict(
            xaxis=dict(title='Composante 1'),
            yaxis=dict(title='Composante 2'),
            zaxis=dict(title='Composante 3') if n_components == 3 else None
        ),
        xaxis=dict(title='Composante 1') if n_components == 2 else None,
        yaxis=dict(title='Composante 2') if n_components == 2 else None,
        showlegend=False
    )
    
    # Créer et afficher la figure
    fig = go.Figure(data=[trace], layout=layout)
    fig.show()
    return unique_words

# Exemple d'utilisation
display_dimred_neighbors(model, ['chat', 'chien'], n_components=2, topn=5, method='umap')

c:\Users\Julien\miniconda3\envs\PCD\Lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



['chien',
 'minou',
 'petit_chatte',
 'aboyer',
 'chat',
 'toutou',
 'chiens',
 'matou',
 'cocker',
 'chienne',
 'chatte',
 'miauler']

## 4. Application à l'étude des mots voisins dans word2vec

Veuillez choisir quatre mots (`word_list = [m1, m2, m3, m4]`) de façon à ce que m1 et m2 soient très proches par leur sens ou leur usage, m3 un peu plus éloigné, et m4 très éloigné.  L'objectif de cette question est d'étudier la distribution des mots voisins de ces quatre mots (entre 10 et 50) par une visualisation en 2D ou en 3D des vecteurs de mots.  

En expérimentant avec plusieurs configurations (méthode de réduction de dimensionnalité, valeur de `n_neighbors`, nombre de voisins `topn`), veuillez répondre aux questions suivantes, à l'aide d'un ou plusieurs graphiques par question :

**a.** si on utilise une méthode de réduction de dimensionnalité linéaire (PCA ou MDS métrique), les quatre groupes de mots sont-ils clairement séparés dans word2vec ?

**b.** si on utilise une méthode de réduction de dimensionnalité non-linéaire (Isomap, t-SNE ou UMAP), peut-on mieux mettre en évidence les quatre groupes de mots ?  Quels sont les meilleurs paramètres que vous avez trouvés permettant de bien identifier les quatre ensembles ?

**c.** inversement, trouvez-vous des paramètres qui aboutissent à cinq clusters ? ou à trois ?

In [43]:
# Insérer ici la liste de mots.
mots = ['grecque', 'latin', 'oublier', 'nourriture']

In [148]:
# Question 4a
display_dimred_neighbors(model, mots, n_components=3, topn=10, method='pca', n_neighbors=10)
display_dimred_neighbors(model, mots, n_components=3, topn=10, method='mds', n_neighbors=10)
#
# En utilisant la méthode PCA, on peut voir que les mots 'latin' et 'grecque' sont proches l'un de l'autre (meme cluster)
# Nous observons également un cluster différent pour 'nourriture' et 'oublier'
# En revanche pour la méthode MDS, les mots 'latin' et 'grecque' sont plus éloignés l'un de l'autre ainsi que la densité en plus homogène, les clusters ne sont plus évidents de manière visuelle.
# A noter que la position des différents cluster reste la même pour les deux méthodes.



['dire',
 'nouriture',
 'là',
 'nourrir',
 'jamais',
 'hellènes',
 'ration_quotidien',
 'ne',
 'gréco-',
 'latin_médiéval',
 'greque',
 've_siècle_av_jésus-christ',
 'ionien',
 'grecquer',
 'falloir',
 'oublier',
 'vivres',
 'grecs',
 'hellénique',
 'iiie_siècle_av_jésus-christ',
 'mot_latin',
 'proie_vivant',
 'latiniser',
 'pâtée',
 'langue_vulgaire',
 'rien',
 'étymologie',
 'racine_grec',
 'grammairien',
 'grec',
 'verbe_latin',
 'affamer',
 'nourriture',
 'aliment',
 'mais',
 'nourriture_sain',
 'manger',
 'latiniste',
 'pas',
 'latin',
 'toujours',
 'grecque',
 'quand']

In [ ]:
# Question 4b
display_dimred_neighbors(model, mots, n_components=3, topn=10, method='tsne', n_neighbors=3)
display_dimred_neighbors(model, mots, n_components=3, topn=10, method='isomap', n_neighbors=3)
display_dimred_neighbors(model, mots, n_components=3, topn=10, method='umap', n_neighbors=3)
# Avec la méthode TSNE, on observe que les clusters sont très proches les uns des autres, même les mots dont la distance sémantique est grande.
# Avec la méthode Isomap, on observe que les clusters sont plus éloignés les uns des autres, mais les mots 'latin' et 'grecque' sont plus proches l'un de l'autre. Nous observons
# également un 'point' de contact entre les 3 mots.
# Avec la méthode UMAP, on observe que les clusters sont plus éloignés les uns des autres, nous obtenons de vrais clusters distincts pour chaque mot.

c:\Users\Julien\miniconda3\envs\PCD\Lib\site-packages\sklearn\manifold\_t_sne.py:1162: FutureWarning:

'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.



c:\Users\Julien\miniconda3\envs\PCD\Lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



['dire',
 'nouriture',
 'là',
 'nourrir',
 'jamais',
 'hellènes',
 'ration_quotidien',
 'ne',
 'gréco-',
 'latin_médiéval',
 'greque',
 've_siècle_av_jésus-christ',
 'ionien',
 'grecquer',
 'falloir',
 'oublier',
 'vivres',
 'grecs',
 'hellénique',
 'iiie_siècle_av_jésus-christ',
 'mot_latin',
 'proie_vivant',
 'latiniser',
 'pâtée',
 'langue_vulgaire',
 'rien',
 'étymologie',
 'racine_grec',
 'grammairien',
 'grec',
 'verbe_latin',
 'affamer',
 'nourriture',
 'aliment',
 'mais',
 'nourriture_sain',
 'manger',
 'latiniste',
 'pas',
 'latin',
 'toujours',
 'grecque',
 'quand']

In [ ]:
# Question 4c
display_dimred_neighbors(model, mots, n_components=3, topn=20, method='umap', n_neighbors=10)
# obtenons un cluster plus grand pour les mots 'latin' et 'grecque' avec cette méthode. Nous pourrions considérer que celui ci forme deux sous cluster distincts.
# Par exemple des noms d'auteurs et des concepts grammaticaux
# 

c:\Users\Julien\miniconda3\envs\PCD\Lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



['dire',
 'excrément',
 'là',
 'nourrir',
 'jamais',
 'lui',
 'régime_alimentaire',
 'ne',
 'vouloir',
 'latin_médiéval',
 've_siècle_av_jésus-christ',
 'ionien',
 'achille_tatius',
 'grecquer',
 'proie_vivant',
 'iiie_siècle_av_jésus-christ',
 'hellénistique',
 'latiniser',
 'pâtée',
 'époque_mycénien',
 'etymologie',
 'affamer',
 'verbe_latin',
 'aliment',
 'car',
 'latiniste',
 'origine_germanique',
 'toujours',
 'nouriture',
 'penser',
 'ration_quotidien',
 'gréco-',
 'peut-être',
 'caput',
 'nourriture_abondant',
 'racine_indo-_européen',
 'vivres',
 'hellénique',
 'nourriture',
 'héraclée',
 'mais',
 'pas',
 'latin',
 'grecques',
 'perses',
 'quand',
 'juste',
 'greque',
 'victuaille',
 'nourrir_exclusivement',
 'tellement',
 'rien',
 'étymologie',
 'racine_grec',
 'grec',
 'dorien',
 'croire',
 'heureusement',
 'frugal',
 'poème_homérique',
 'sinon',
 'grecque',
 'ménandre',
 'hellènes',
 'clitophon',
 'ration_alimentaire',
 'denrée',
 'gréco-_latin',
 'gaffiot',
 'traduction_la

## 5. Outil d'assistance pour le jeu Cémantix

Le jeu en ligne [Cémantix](https://cemantix.certitudes.org/) (aussi proposé sur le site [Dictaly](https://www.dictaly.com/semantiques/))  demande de deviner le *mot du jour* en l'approchant peu à peu par des mots candidats.  Pour chacun, le système indique pour sa similarité word2vec avec le *mot du jour*, ce qui permet de se rapprocher graduellement de la solution.  Expérimentez d'abord un court instant avec le jeu 😀.

L'objectif de cette question est de visualiser les voisinages de mots, pour vous aider à proposer des mots candidats et trouver plus vite la solution.  La procédure est la suivante :
* essayez trois mots au hasard dans Cémantix
* affichez 20-30 mots voisins de ces trois mots grâce à la fonction `display_dimred_neighbors`
* à l'aide des mots affichés, essayez des mots candidats dans Cémantix
* changez l'affichage en remplaçant les trois mots par de meilleurs mots
* continuez jusqu'à trouver le *mot du jour*

**Notes**
* si vous le souhaitez, vous pouvez utiliser le modèle word2vec identique à celui de Cémantix, qui est frWac_no_postag_phrase_500_cbow_cut10.bin fourni par [J.-Ph. Fauconnier](https://fauconnier.github.io/#data)
* ce modèle fait environ 2.3 Go et contient aussi des expressions de plusieurs mots (séparés par '_')
* vous pouvez choisir d'utiliser le petit ou le grand modèle, sachant qu'avec le premier, les suggestions sont moins pertinentes
* Cémantix ignore les expressions de plusieurs mots présentes dans le grand modèle -- vous pouvez choisir de le faire ou non.

In [165]:
# candidate_words = ['homme', 'rouge', 'étudier']
candidate_words = ['salle', 'conférence', 'espace']
m = display_dimred_neighbors(model, candidate_words, n_components=3, topn=20, method='pca', n_neighbors=5)


In [166]:
'atelier' in m
# afficher les voisin de atelier
m

['salles',
 'spatial',
 'paysage',
 'rez-de-chaussée',
 'réfectoire',
 'conférencier',
 'salle_attenant',
 'lieux',
 'conférence',
 'urbain',
 'amphithéâtre',
 'privilégier',
 'er_étage',
 'table-ronde',
 'vestiaire_douche',
 'conférence_plénier',
 'séminaire',
 'dimension',
 'conference',
 'espaces',
 'hall',
 'espace_interstitiel',
 'culturel',
 'coorganiser',
 'conférence_débattre',
 'organiser_conjointement',
 'espace',
 'gymnase',
 'séance_inaugural',
 'élément_structurant',
 'cafétéria',
 'spatialité',
 'spatialement',
 'dimension_spatial',
 'colloque',
 'table_rond',
 'salle_modulable',
 'symposium',
 'aménager',
 'espace_privatif',
 'espace_scénique',
 'paysager',
 'conférence-débat',
 'rotonde',
 'trame_urbain',
 'salle',
 'étage',
 'conférencier_inviter',
 'cheminement_piéton',
 'conferences',
 'tables-ronde',
 'table_chaise',
 'locaux',
 'conférences',
 'conférence_inaugural',
 'vaste_hall',
 'cafeteria',
 'tissu_urbain',
 'conférences-débat',
 'co-_organiser',
 'salle_polyv

Veuillez noter ici vos observations sur la procédure, et coller un extrait montrant votre meilleure performance au jeu.

La visualisation des voisions en pca nous aide à démarrer le jeu avec très peu de mots découverts.

![image.png](c.jpg)

**Fin du Labo.**  Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.